In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%cd '/content/drive/My Drive/DigiKala'

/content/drive/My Drive/DigiKala


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [25]:
df_train = pd.read_csv('train_users.csv')
df_test = pd.read_csv('test_users.csv')

In [26]:
train = df_train
train.fillna(' ', inplace=True)

train['content'] = train['title'] + \
                   train['comment'] + \
                   train['advantages'] + \
                   train['disadvantages'] + \
                   train['title_fa_product'] + \
                   train['title_fa_category']

train = train[['content', 'rate']]
train.head()

,content,rate
0,مثل بقیه محصولات الکل دار پوست رو خشک نمیکنه ...,100.0
1,Redmi 8با این مبلغ اگه امکانات و ارزش خرید واس...,80.0
2,عالیخوبه فقط کج و کوله بدستم رسید ولی پسرم خیل...,100.0
3,در کل خوب بود ولی متاسفانه درب محصول شکسته بو...,70.0
4,من که خیلی باهاش حال کردم فقط من که همیشه L م...,72.0


In [27]:
test = df_test
test.fillna(' ', inplace=True)

test['content'] = test['title'] + \
                  test['comment'] + \
                  test['advantages'] + \
                  test['disadvantages'] + \
                  test['title_fa_product'] + \
                  test['title_fa_category']

test = test[['content']]
test.head()

,content
0,با کیفیتبسته بندی بهداشتی و کیفیت عالی گوش پا...
1,"گوشی موبایل شیائومیاین گوشی فوق‌العاده است[""ای..."
2,صداش قطع میشه باتریشم خیییلی ضعیفه هدفون بی ...
3,بنظرم‌ خوبه خمیر دندان نسیم مدل Feluoride Pr...
4,"عااالیهعااالیه[""عاااالیه""] کرم موبر صورت و بدن..."


In [28]:
X_train = train['content']
y_train = train['rate']
X_test = test['content']

In [29]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [30]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,128,input_length=max_len)(inputs)
  layer = Bidirectional(LSTM(64, return_sequences=True))(layer)
  layer = Bidirectional(LSTM(64))(layer)
  layer = Dense(128,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [31]:
model = RNN()
model.summary()
model.compile(loss = 'mean_squared_error',
              optimizer = RMSprop(),
              metrics = ['mean_squared_error'])

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 128)          128000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 150, 128)          98816     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [32]:
model.fit(sequences_matrix,
          y_train,
          batch_size=128,
          epochs=10,
          validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

Epoch 1/10
1653/1653 [==============================] - 64s 39ms/step - loss: 1371.7225 - mean_squared_error: 1371.7225 - val_loss: 1236.4459 - val_mean_squared_error: 1236.4459
Epoch 2/10
1653/1653 [==============================] - 63s 38ms/step - loss: 1253.5713 - mean_squared_error: 1253.5713 - val_loss: 1189.7433 - val_mean_squared_error: 1189.7433
Epoch 3/10
1653/1653 [==============================] - 62s 38ms/step - loss: 1220.0085 - mean_squared_error: 1220.0085 - val_loss: 1166.5967 - val_mean_squared_error: 1166.5967
Epoch 4/10
1653/1653 [==============================] - 63s 38ms/step - loss: 1194.6061 - mean_squared_error: 1194.6061 - val_loss: 1161.3578 - val_mean_squared_error: 1161.3578
Epoch 5/10
1653/1653 [==============================] - 62s 38ms/step - loss: 1175.4602 - mean_squared_error: 1175.4602 - val_loss: 1141.8586 - val_mean_squared_error: 1141.8586
Epoch 6/10
1653/1653 [==============================] - 62s 38ms/step - loss: 1158.8724 - mean_squared_error: 

In [33]:
txts = tok.texts_to_sequences(X_test)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)

In [34]:
y_test = preds.tolist()

df_final = pd.DataFrame(y_test)
df_final.head()

,0
0,66.310501
1,57.560089
2,48.127178
3,64.667015
4,54.180367


In [36]:
df_final.to_csv('rate_status.csv', index = True, index_label=['id'], header=['rate'])